In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from scipy.integrate import solve_ivp

## ODE System 
1. This system has a combined intrinsic and emergent growth rate AND competition.

In [2]:
def dNdt(t, N, rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                   #benthos
         ρB, hB, ηB, sB, φB, σB, θB, eB, pB, aBB, aBG, aBS,                  #browsers
         ρG, hG, ηG, sG, φG, σG, θG, eG, pG, aGB, aGG, aGS,                  #grazers
         ρS, hS, ηS, sS, φS, σS, θS, eS, pS, aSB, aSG, aSS):                 #scrapers
    
    C,M,T,B,G,S = N
   
    dC = (rC*C*(1-C-M-T) + αCT*rC*C*T - αMC*rM*C*M - μC*C)*dt
    dM = (rM*M*(1-C-M-T) + αMT*rM*T*M + αMC*rM*C*M - μM*M - ((hB*B*M)/((hB*ηB*M)+1)))*dt
    dT = (rT*T*(1-C-M-T) - αCT*rC*C*T - αMT*rM*T*M - μT*T - ((hG*G*T)/((hG*ηG*T)+1)) - ((hS*S*T)/((hS*ηS*T)+1)))*dt

    B_comp = B*(aBB*B + aBG*G + aBS*S)
    G_comp = G*(aGB*B + aGG*G + aGS*S)
    S_comp = S*(aSB*B + aSG*G + aSS*S)
    
    dB = (((sB + ρB*(φB*hB*B*M))*(1-((B+B_comp)/((1-σB) + (σB*C))))) + θB*B - eB*B - pB*B)*dt
    dG = (((sG + ρG*(φG*hG*G*T))*(1-((G+G_comp)/((1-σG) + (σG*C))))) + θG*G - eG*G - pG*G)*dt
    dS = (((sS + ρS*(φS*hS*S*T))*(1-((S+S_comp)/((1-σS) + (σS*C))))) + θS*S - eS*S - pS*S)*dt

    return dC, dM, dT, dB, dG, dS

## Initializing Timesteps

1. In these runs, the initial benthic conditions promote AltSS.

In [3]:
dt = 1
NUMSTEPS = 10000 #you should eventually add a term that terminates solving after reaching equilibrium
NUMYEARS = int(NUMSTEPS/dt)
time_points = np.linspace(0, NUMYEARS, NUMSTEPS+1)

## Defining Pollutant Arrays

In [4]:
nutr_value = np.linspace(0, 100, 21) #you increase this from 10x10 to 20x20
nutr_max = 100

sed_value = np.linspace(0, 100, 21)
sed_max = 100

## Defining function to locate AltSS

1. Each pollutant v. model component association has it's own association strength, "x_," so that we can piece apart effects of each association, then group them by direct v. indirect and nutrients v. sediments.

In [6]:
def sed_nutr_AltSS(scenario, x): 
    
    outputs = []
    outputs_prime = []
    
    for sed in sed_value:
        print(sed)
    
        for nutr in nutr_value:
        
            for C0 in np.linspace(0.01,0.99,2):
            
                B0, G0, S0 = 0.1, 0.1, 0.1
                C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2)
     
                #COMPETITION ---------------------------------------------------------------------
                #----------------------------------------------------------------------------------
                
                aBB, aBG, aBS = 0.5, 0.25, 0.25
                aGB, aGG, aGS = 0.25, 0.5, 0.25
                aSB, aSG, aSS = 0.25, 0.25, 0.5

                #ASSOCIATIONS ---------------------------------------------------------------------
                #----------------------------------------------------------------------------------
                
                xc, xm, xt, xh, xe = x, x, x, x, x
                 
                #Benthos------------------------------------------------
                # Sediment v. Coral Mortality Rate Association
                μC_min = 0.02
                μC_max = 0.1
                μC = μC_max + (μC_min - μC_max)*(((sed_max-sed)/100)**xc)
                
                # Nutrient v. Macroalgae Growth Rate Association
                rM_min = 0.05
                rM_max = 0.4
                rM = rM_max + (rM_min - rM_max)*(((nutr_max-nutr)/100)**xm)
                
                # Nutrient v. Turf Algae Growth Rate Association
                rT_min = 2
                rT_max = 20
                rT = rT_max + (rT_min - rT_max)*(((nutr_max-nutr)/100)**xt)
                
                #Herbivores------------------------------------------------
                # Sediment v. Herbivory Rate Association
                h_min = 0
                h_max = 1
                h = h_min + (h_max - h_min)*(((sed_max-sed)/100)**xh)
                
                # Sediment v. Emigration Rate Association
                e_min = 0
                e_max = 0.9
                e = e_max + (e_min - e_max)*(((sed_max-sed)/100)**xe)
        
                
                #BENTHOS --------------------------------------------------------------------------
                #----------------------------------------------------------------------------------
        
                μM_AltSS_12 = 0.12
                rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, rM, rT, 0.25, 0.9, 0.1, μC, μM_AltSS_12, 10
    
                                
                #HERBIVORES------------------------------------------------------------------------
                #----------------------------------------------------------------------------------
    
                #Baseline Benthos (No Herbivores)------------------------------
                if scenario == 'baselinebenthos':
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB = 0, 1, 0, 0, 0, 0, 0, 0, 0
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = 0, 1, 0, 0, 0, 0, 0, 0, 0
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Browsers-----------------------------------------------------
                elif scenario == 'browser':
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB, = h, 1, 0.12, 1, 0, e, 0.05, 1, 0.02
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = 0, 1, 0, 0, 0, 0, 0, 0, 0
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = 0, 1, 0, 0, 0, 0, 0, 0, 0

                #Grazers-----------------------------------------------------
                elif scenario == 'grazer':
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = h, 1, 0.30, 1, 0, e, 0.05, 2.48, 0.02
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB = 0, 1, 0, 0, 0, 0, 0, 0, 0
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Scrapers-----------------------------------------------------
                elif scenario == 'scraper':
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = h, 1, 0.37, 1, 0, e, 0.05, 3.02, 0.02
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB = 0, 1, 0, 0, 0, 0, 0, 0, 0
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Browsers & Grazers-------------------------------------------
                elif scenario == 'browser_grazer':
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB, = h, 1, 0.12, 1, 0, e, 0.05, 1, 0.02
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = h, 1, 0.30, 1, 0, e, 0.05, 2.48, 0.02
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Browsers & Scrapers------------------------------------------
                elif scenario == 'browser_scraper':
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB = h, 1, 0.12, 1, 0, e, 0.05, 1, 0.02
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = h, 1, 0.37, 1, 0, e, 0.05, 3.02, 0.02
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Grazers & Scrapers------------------------------------------
                elif scenario == 'grazer_scraper':
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = h, 1, 0.30, 1, 0, e, 0.05, 2.48, 0.02
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = h, 1, 0.37, 1, 0, e, 0.05, 3.02, 0.02
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB = 0, 1, 0, 0, 0, 0, 0, 0, 0
    
                #Browsers, Grazers, & Scrapers--------------------------------
                elif scenario == 'all_herbivores':
                    hB, ηB, sB, σB, θB, eB, pB, ρB, φB, = h, 1, 0.12, 1, 0, e, 0.05, 1, 0.02
                    hG, ηG, sG, σG, θG, eG, pG, ρG, φG = h, 1, 0.30, 1, 0, e, 0.05, 2.48, 0.02
                    hS, ηS, sS, σS, θS, eS, pS, ρS, φS = h, 1, 0.37, 1, 0, e, 0.05, 3.02, 0.02
                
                sol = solve_ivp(dNdt, [0,NUMYEARS], [C0, M0, T0, B0, G0, S0], 
                                method = 'RK45', args = 
                                (rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                                     #benthos
                                hB, ηB, sB, σB, θB, eB, pB, ρB, φB, aBB, aBG, aBS,                          #browsers
                                hG, ηG, sG, σG, θG, eG, pG, ρG, φG, aGB, aGG, aGS,                          #grazers
                                hS, ηS, sS, σS, θS, eS, pS, ρS, φS, aSB, aSG, aSS),                         #scrapers
                                dense_output=True)
    
                N = sol.sol(time_points)
    
                C_array = N[0,:]
                M_array = N[1,:]
                T_array = N[2,:]
                B_array = N[3,:]
                G_array = N[4,:]
                S_array = N[5,:]

            
                F_prime = 1-(C_array[-1] + M_array[-1] + T_array[-1])
    
                if C_array[-1] >= 0.01 and M_array[-1] < 0.01 and T_array[-1] < 0.01:
                    color_col = 'allC'
                
                elif M_array[-1] >= 0.01 and C_array[-1] < 0.01 and T_array[-1] < 0.01:
                    color_col = 'allM'
                
                elif T_array[-1] >= 0.01 and C_array[-1] < 0.01 and M_array[-1] < 0.01:
                    color_col = 'allT'
                
                elif C_array[-1] < 0.01 and M_array[-1] < 0.01 and T_array[-1] < 0.01:
                    color_col = 'allF'
                    
                elif C_array[-1] >= 0.01 and M_array[-1] >= 0.01 and T_array[-1] >= 0.01 and C_array[-1] > M_array[-1] and C_array[-1] > T_array[-1] and C_array[-1] > F_prime:
                    color_col = 'mostlyC'
                
                elif C_array[-1] >= 0.01 and M_array[-1] >= 0.01 and T_array[-1] < 0.01 and C_array[-1] > M_array[-1] and C_array[-1] > T_array[-1] and C_array[-1] > F_prime:
                    color_col = 'mostlyC_noT'
                
                elif M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] >= 0.01 and M_array[-1] > C_array[-1] and M_array[-1] > T_array[-1] and M_array[-1] > F_prime:
                    color_col = 'mostlyM'
                
                elif M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] < 0.01 and M_array[-1] > C_array[-1] and M_array[-1] > T_array[-1] and M_array[-1] > F_prime:
                    color_col = 'mostlyM_noT'
                
                elif T_array[-1] >= 0.01 and M_array[-1] >= 0.01 and C_array[-1] >= 0.01 and T_array[-1] > C_array[-1] and T_array[-1] > M_array[-1] and T_array[-1] > F_prime:
                    color_col = 'mostlyT'
    
                else:
                    color_col = 'other'

                outputs.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                                          #benthos
                                                hB, ηB, sB, σB, θB, eB, pB, ρB, φB, aBB, aBG, aBS,              #browsers
                                                hG, ηG, sG, σG, θG, eG, pG, ρG, φG, aGB, aGG, aGS,              #grazers
                                                hS, ηS, sS, σS, θS, eS, pS, ρS, φS, aSB, aSG, aSS,              #scrapers
                                                xc, xm, xt, xh, xe, sed, nutr,                                  #associations
                                                C_array, M_array, T_array, B_array, G_array, S_array,
                                                color_col))
                                    
                outputs_prime.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                                    #benthos
                                hB, ηB, sB, σB, θB, eB, pB, ρB, φB, aBB, aBG, aBS,                              #browsers
                                hG, ηG, sG, σG, θG, eG, pG, ρG, φG, aGB, aGG, aGS,                              #grazers
                                hS, ηS, sS, σS, θS, eS, pS, ρS, φS, aSB, aSG, aSS,                              #scrapers
                                xc, xm, xt, xh, xe, sed, nutr,                                                  #associations
                                C_array[-1], M_array[-1], T_array[-1], F_prime, B_array[-1], G_array[-1], S_array[-1],
                                color_col))

    df_timeseries = pd.DataFrame(outputs, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT',                 #benthos
                                            'hB', 'ηB', 'sB', 'σB', 'θB', 'eB', 'pB', 'ρB', 'φB', 'aBB', 'aBG', 'aBS',      #browsers
                                            'hG', 'ηG', 'sG', 'σG', 'θG', 'eG', 'pG', 'ρG', 'φG', 'aGB', 'aGG', 'aGS',      #grazers
                                            'hS', 'ηS', 'sS', 'σS', 'θS', 'eS', 'pS', 'ρS', 'φS', 'aSB', 'aSG', 'aSS',      #scrapers
                                            'xc', 'xm', 'xt', 'xh', 'xe', 'sed', 'nutr',                                    #associations
                                            'C_array', 'M_array', 'T_array', 'B_array', 'G_array', 'S_array',
                                            'color_col'])
        
        
    df_prime = pd.DataFrame(outputs_prime, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT',                #benthos
                                            'hB', 'ηB', 'sB', 'σB', 'θB', 'eB', 'pB', 'ρB', 'φB', 'aBB', 'aBG', 'aBS',      #browsers
                                            'hG', 'ηG', 'sG', 'σG', 'θG', 'eG', 'pG', 'ρG', 'φG', 'aGB', 'aGG', 'aGS',      #grazers
                                            'hS', 'ηS', 'sS', 'σS', 'θS', 'eS', 'pS', 'ρS', 'φS', 'aSB', 'aSG', 'aSS',      #scrapers
                                            'xc', 'xm', 'xt', 'xh', 'xe', 'sed', 'nutr',                                    #associations
                                            'C_prime', 'M_prime', 'T_prime', 'F_prime','B_prime', 'G_prime', 'S_prime', 
                                            'color_col'])
          
    return(df_timeseries, df_prime)

# x = 0 (constant)

In [18]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browser_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'browser', x = 0)

Browser_sedvnutr_x0_053025[0].to_pickle('timeseries_Browser_sedvnutr_x0_053025')
Browser_sedvnutr_x0_053025[1].to_pickle('equilibria_Browser_sedvnutr_x0_053025')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'grazer', x = 0)

Grazer_sedvnutr_x0_053025[0].to_pickle('timeseries_Grazer_sedvnutr_x0_053025')
Grazer_sedvnutr_x0_053025[1].to_pickle('equilibria_Grazer_sedvnutr_x0_053025')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'scraper', x = 0)

Scraper_sedvnutr_x0_053025[0].to_pickle('timeseries_Scraper_sedvnutr_x0_053025')
Scraper_sedvnutr_x0_053025[1].to_pickle('equilibria_Scraper_sedvnutr_x0_053025')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'browser_grazer', x = 0)

BrowserGrazer_sedvnutr_x0_053025[0].to_pickle('timeseries_BrowserGrazer_sedvnutr_x0_053025')
BrowserGrazer_sedvnutr_x0_053025[1].to_pickle('equilibria_BrowserGrazer_sedvnutr_x0_053025')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'browser_scraper', x = 0)

BrowserScraper_sedvnutr_x0_053025[0].to_pickle('timeseries_BrowserScraper_sedvnutr_x0_053025')
BrowserScraper_sedvnutr_x0_053025[1].to_pickle('equilibria_BrowserScraper_sedvnutr_x0_053025')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'grazer_scraper', x = 0)

GrazerScraper_sedvnutr_x0_053025[0].to_pickle('timeseries_GrazerScraper_sedvnutr_x0_053025')
GrazerScraper_sedvnutr_x0_053025[1].to_pickle('equilibria_GrazerScraper_sedvnutr_x0_053025')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'all_herbivores', x = 0)

AllHerbivores_sedvnutr_x0_053025[0].to_pickle('timeseries_AllHerbivores_sedvnutr_x0_053025')
AllHerbivores_sedvnutr_x0_053025[1].to_pickle('equilibria_AllHerbivores_sedvnutr_x0_053025')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_sedvnutr_x0_053025  = sed_nutr_AltSS(scenario = 'baselinebenthos', x = 0)

BaselineBenthos_sedvnutr_x0_053025[0].to_pickle('timeseries_BaselineBenthos_sedvnutr_x0_053025')
BaselineBenthos_sedvnutr_x0_053025[1].to_pickle('equilibria_BaselineBenthos_sedvnutr_x0_053025')


0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0


# x = 1 (linear)

In [31]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browser_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'browser', x = 1)

Browser_sedvnutr_x1_053025[0].to_pickle('timeseries_Browser_sedvnutr_x1_053025')
Browser_sedvnutr_x1_053025[1].to_pickle('equilibria_Browser_sedvnutr_x1_053025')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'grazer', x = 1)

Grazer_sedvnutr_x1_053025[0].to_pickle('timeseries_Grazer_sedvnutr_x1_053025')
Grazer_sedvnutr_x1_053025[1].to_pickle('equilibria_Grazer_sedvnutr_x1_053025')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'scraper', x = 1)

Scraper_sedvnutr_x1_053025[0].to_pickle('timeseries_Scraper_sedvnutr_x1_053025')
Scraper_sedvnutr_x1_053025[1].to_pickle('equilibria_Scraper_sedvnutr_x1_053025')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'browser_grazer', x = 1)

BrowserGrazer_sedvnutr_x1_053025[0].to_pickle('timeseries_BrowserGrazer_sedvnutr_x1_053025')
BrowserGrazer_sedvnutr_x1_053025[1].to_pickle('equilibria_BrowserGrazer_sedvnutr_x1_053025')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'browser_scraper', x = 1)

BrowserScraper_sedvnutr_x1_053025[0].to_pickle('timeseries_BrowserScraper_sedvnutr_x1_053025')
BrowserScraper_sedvnutr_x1_053025[1].to_pickle('equilibria_BrowserScraper_sedvnutr_x1_053025')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'grazer_scraper', x = 1)

GrazerScraper_sedvnutr_x1_053025[0].to_pickle('timeseries_GrazerScraper_sedvnutr_x1_053025')
GrazerScraper_sedvnutr_x1_053025[1].to_pickle('equilibria_GrazerScraper_sedvnutr_x1_053025')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'all_herbivores', x = 1)

AllHerbivores_sedvnutr_x1_053025[0].to_pickle('timeseries_AllHerbivores_sedvnutr_x1_053025')
AllHerbivores_sedvnutr_x1_053025[1].to_pickle('equilibria_AllHerbivores_sedvnutr_x1_053025')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_sedvnutr_x1_053025  = sed_nutr_AltSS(scenario = 'baselinebenthos', x = 1)

BaselineBenthos_sedvnutr_x1_053025[0].to_pickle('timeseries_BaselineBenthos_sedvnutr_x1_053025')
BaselineBenthos_sedvnutr_x1_053025[1].to_pickle('equilibria_BaselineBenthos_sedvnutr_x1_053025')


0.0


KeyboardInterrupt: 

# x = 3 (exponential)

In [7]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browser_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'browser', x = 3)

Browser_sedvnutr_x3_053025[0].to_pickle('timeseries_Browser_sedvnutr_x3_053025')
Browser_sedvnutr_x3_053025[1].to_pickle('equilibria_Browser_sedvnutr_x3_053025')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'grazer', x = 3)

Grazer_sedvnutr_x3_053025[0].to_pickle('timeseries_Grazer_sedvnutr_x3_053025')
Grazer_sedvnutr_x3_053025[1].to_pickle('equilibria_Grazer_sedvnutr_x3_053025')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'scraper', x = 3)

Scraper_sedvnutr_x3_053025[0].to_pickle('timeseries_Scraper_sedvnutr_x3_053025')
Scraper_sedvnutr_x3_053025[1].to_pickle('equilibria_Scraper_sedvnutr_x3_053025')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'browser_grazer', x = 3)

BrowserGrazer_sedvnutr_x3_053025[0].to_pickle('timeseries_BrowserGrazer_sedvnutr_x3_053025')
BrowserGrazer_sedvnutr_x3_053025[1].to_pickle('equilibria_BrowserGrazer_sedvnutr_x3_053025')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'browser_scraper', x = 3)

BrowserScraper_sedvnutr_x3_053025[0].to_pickle('timeseries_BrowserScraper_sedvnutr_x3_053025')
BrowserScraper_sedvnutr_x3_053025[1].to_pickle('equilibria_BrowserScraper_sedvnutr_x3_053025')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'grazer_scraper', x = 3)

GrazerScraper_sedvnutr_x3_053025[0].to_pickle('timeseries_GrazerScraper_sedvnutr_x3_053025')
GrazerScraper_sedvnutr_x3_053025[1].to_pickle('equilibria_GrazerScraper_sedvnutr_x3_053025')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'all_herbivores', x = 3)

AllHerbivores_sedvnutr_x3_053025[0].to_pickle('timeseries_AllHerbivores_sedvnutr_x3_C99_062725')
AllHerbivores_sedvnutr_x3_053025[1].to_pickle('equilibria_AllHerbivores_sedvnutr_x3_C99_062725')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_sedvnutr_x3_053025  = sed_nutr_AltSS(scenario = 'baselinebenthos', x = 3)

BaselineBenthos_sedvnutr_x3_053025[0].to_pickle('timeseries_BaselineBenthos_sedvnutr_x3_053025')
BaselineBenthos_sedvnutr_x3_053025[1].to_pickle('equilibria_BaselineBenthos_sedvnutr_x3_053025')


0.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0
55.0
60.0
65.0
70.0
75.0
80.0
85.0
90.0
95.0
100.0
0.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0
55.0
60.0
65.0
70.0
75.0
80.0
85.0
90.0
95.0
100.0


# x = 5 (exponential)

In [4]:
#Browsers-----------------------------------
B0, G0, S0 = 0.1, 0, 0

Browser_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'browser', x = 5)

Browser_sedvnutr_x5_053025[0].to_pickle('timeseries_Browser_sedvnutr_x5_053025')
Browser_sedvnutr_x5_053025[1].to_pickle('equilibria_Browser_sedvnutr_x5_053025')

#Grazers-----------------------------------
B0, G0, S0 = 0, 0.1, 0

Grazer_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'grazer', x = 5)

Grazer_sedvnutr_x5_053025[0].to_pickle('timeseries_Grazer_sedvnutr_x5_053025')
Grazer_sedvnutr_x5_053025[1].to_pickle('equilibria_Grazer_sedvnutr_x5_053025')

#Scrapers-----------------------------------
B0, G0, S0 = 0, 0, 0.1

Scraper_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'scraper', x = 5)

Scraper_sedvnutr_x5_053025[0].to_pickle('timeseries_Scraper_sedvnutr_x5_053025')
Scraper_sedvnutr_x5_053025[1].to_pickle('equilibria_Scraper_sedvnutr_x5_053025')

#Browsers & Grazers-------------------------
B0, G0, S0 = 0.1, 0.1, 0

BrowserGrazer_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'browser_grazer', x = 5)

BrowserGrazer_sedvnutr_x5_053025[0].to_pickle('timeseries_BrowserGrazer_sedvnutr_x5_053025')
BrowserGrazer_sedvnutr_x5_053025[1].to_pickle('equilibria_BrowserGrazer_sedvnutr_x5_053025')

#Browsers & Scrapers-------------------------
B0, G0, S0 = 0.1, 0, 0.1

BrowserScraper_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'browser_scraper', x = 5)

BrowserScraper_sedvnutr_x5_053025[0].to_pickle('timeseries_BrowserScraper_sedvnutr_x5_053025')
BrowserScraper_sedvnutr_x5_053025[1].to_pickle('equilibria_BrowserScraper_sedvnutr_x5_053025')

#Grazers & Scrapers-------------------------
B0, G0, S0 = 0, 0.1, 0.1

GrazerScraper_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'grazer_scraper', x = 5)

GrazerScraper_sedvnutr_x5_053025[0].to_pickle('timeseries_GrazerScraper_sedvnutr_x5_053025')
GrazerScraper_sedvnutr_x5_053025[1].to_pickle('equilibria_GrazerScraper_sedvnutr_x5_053025')

#All Herbivores-------------------------
B0, G0, S0 = 0.1, 0.1, 0.1

AllHerbivores_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'all_herbivores', x = 5)

AllHerbivores_sedvnutr_x5_053025[0].to_pickle('timeseries_AllHerbivores_sedvnutr_x5_053025')
AllHerbivores_sedvnutr_x5_053025[1].to_pickle('equilibria_AllHerbivores_sedvnutr_x5_053025')

#Baseline Benthos-------------------------
B0, G0, S0 = 0, 0, 0

BaselineBenthos_sedvnutr_x5_053025  = sed_nutr_AltSS(scenario = 'baselinebenthos', x = 5)

BaselineBenthos_sedvnutr_x5_053025[0].to_pickle('timeseries_BaselineBenthos_sedvnutr_x5_053025')
BaselineBenthos_sedvnutr_x5_053025[1].to_pickle('equilibria_BaselineBenthos_sedvnutr_x5_053025')